In [1]:
# install MeCab
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
# install NEologd
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a
# install neologdn
!pip install neologdn
# install emoji
!pip install emoji
# upgrade tqdm
!pip install tqdm --upgrade
# install pytorch-lightning
!pip install pytorch-lightning

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp

     |████████████████████████████████| 235kB 2.8MB/s 
     |████████████████████████████████| 829kB 8.4MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=6dd25cc146d587eddeb86c380d82262d14e9188040df26f40e3de25eb8a551c9
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


In [0]:
import math
import numpy as np
import pandas as pd
import random
import MeCab
import re
import neologdn
import string
import emoji
import os
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torchtext
import torch.optim as optim
from torchtext.vocab import Vectors
from sklearn.metrics import accuracy_score
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping

import warnings
warnings.filterwarnings('ignore')

In [0]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/NLP/RionTweetClassifier')
from utils.dataset import get_dataset
from utils.transformer import TransformerClassification, src_mask

In [0]:
def seed_everything(seed=1234):
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

seed_everything(seed=1234)

In [4]:
# dataset
train_ds, test_ds, val_ds, TEXT = get_dataset(max_length=256, split_ratio=[0.92, 0.04, 0.04])

INFO:torchtext.vocab:Loading vectors from .vector_cache/model.vec.pt


In [0]:
class MyLightningModule(pl.LightningModule):
    def __init__(self, model, train_ds, val_ds, test_ds, batch_size):
        super().__init__()
        # dataset path
        self.train_ds = train_ds
        self.val_ds = val_ds
        self.test_ds = test_ds
        self.bs = batch_size
        self.model = model

    def forward(self, x):
        # モデルの順伝搬処理
        input_mask = src_mask(x)
        x = self.model(x, input_mask)

        return x

    def training_step(self, batch, batch_idx):
        # train setのmini-batchにおける処理
        # REQUIRED
        x = batch.Text[0].to(device)
        y = batch.Label.unsqueeze(1).to(device)
        y_hat = self.forward(x)

        criterion = nn.BCEWithLogitsLoss(reduction='sum')
        loss = criterion(y_hat, y)
        tensorboard_logs = {'train_loss': loss.item()}

        return {'loss': loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        # valid setのmini-batchにおける処理
        x = batch.Text[0].to(device)
        y = batch.Label.unsqueeze(1).to(device)
        out = self.forward(x)
        
        bs = len(batch.Label)
        pred = torch.where(out >= 0.5, torch.ones(bs, 1).to(device), torch.zeros(bs, 1).to(device))

        criterion = nn.BCEWithLogitsLoss(reduction='sum')
        loss = criterion(out, y)

        return {'val_loss': loss, 'label': y, 'pred': pred}

    def test_step(self, batch, batch_idx):
        # test setのmini-batchにおける処理
        x = batch.Text[0].to(device)
        y = batch.Label.unsqueeze(1).to(device)
        out = self.forward(x)
        
        bs = len(batch.Label)
        pred = torch.where(out >= 0.5, torch.ones(bs, 1).to(device), torch.zeros(bs, 1).to(device))

        criterion = nn.BCEWithLogitsLoss(reduction='sum')
        loss = criterion(out, y)

        return {'test_loss': loss, 'label': y, 'pred': pred}

    def validation_epoch_end(self, outputs):
        # valid setのmini-batch処理終了時の処理
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        # accuracy
        acc = torch.mean(torch.cat([(output['label'] == output['pred']) * 1.0 for output in outputs]))

        tensorboard_logs = {'val_loss': avg_loss}
        tqdm_dict = {'avg_val_loss': avg_loss, 'val_acc': acc}

        return {'progress_bar': tqdm_dict, 'avg_val_loss': avg_loss, 'log': tensorboard_logs}

    def test_epoch_end(self, outputs):
        # test setのmini-batch処理終了時の処理
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        # accuracy
        acc = torch.mean(torch.cat([(output['label'] == output['pred']) * 1.0 for output in outputs]))

        tensorboard_logs = {'test_loss': avg_loss}
        tqdm_dict = {'avg_test_loss': avg_loss, 'test_acc': acc}

        return {'progress_bar': tqdm_dict, 'avg_val_loss': avg_loss, 'log': tensorboard_logs}

    def configure_optimizers(self):
        # 最適化手法・学習率スケジュールの設定
        # REQUIRED
        optimizer = optim.Adam(self.parameters(), lr=1e-5)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
        
        return [optimizer], [scheduler]

    # @pl.data_loader # 最新版はデコレーターは非推奨
    def train_dataloader(self):
        # REQUIRED
        train_loader = torchtext.data.Iterator(train_ds, batch_size=self.bs, train=True)
        
        return train_loader

    # @pl.data_loader
    def val_dataloader(self):
        # OPTIONAL
        val_loader = torchtext.data.Iterator(val_ds, batch_size=self.bs, train=False, sort=False)
        
        return val_loader

    def test_dataloader(self):
        test_loader = torchtext.data.Iterator(val_ds, batch_size=self.bs, train=False, sort=False)

        return test_loader

# early stopping callback
early_stop_callback = EarlyStopping(min_delta=0.00,
                                    patience=5,
                                    verbose=True,
                                    monitor='val_loss',
                                    mode='min')

In [6]:
net = TransformerClassification(text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=256, heads=6, N=2, output_dim=1)
net

TransformerClassification(
  (embed): Embedder(
    (embeddings): Embedding(3759, 300)
  )
  (pe): PositionalEncoder()
  (layers): ModuleList(
    (0): EncoderLayer(
      (norm_1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
      (norm_2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
      (attn): MultiHeadAttention(
        (q_linear): Linear(in_features=300, out_features=300, bias=True)
        (v_linear): Linear(in_features=300, out_features=300, bias=True)
        (k_linear): Linear(in_features=300, out_features=300, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (out): Linear(in_features=300, out_features=300, bias=True)
      )
      (ff): PositionwiseFeedForward(
        (linear_1): Linear(in_features=300, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear_2): Linear(in_features=2048, out_features=300, bias=True)
      )
      (dropout_1): Dropout(p=0.1, inplace=False)
      (dropout_2): Dropou

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

lightning_model = MyLightningModule(net, train_ds, val_ds, test_ds, batch_size=8)

trainer = Trainer(max_nb_epochs=10, 
                  gpus=1, 
                #   early_stop_callback=early_stop_callback
                  )

trainer.fit(lightning_model)

INFO:lightning:GPU available: True, used: True
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:
   | Name                         | Type                      | Params
-----------------------------------------------------------------------
0  | model                        | TransformerClassification | 4 M   
1  | model.embed                  | Embedder                  | 1 M   
2  | model.embed.embeddings       | Embedding                 | 1 M   
3  | model.pe                     | PositionalEncoder         | 0     
4  | model.layers                 | ModuleList                | 3 M   
5  | model.layers.0               | EncoderLayer              | 1 M   
6  | model.layers.0.norm_1        | LayerNorm                 | 600   
7  | model.layers.0.norm_2        | LayerNorm                 | 600   
8  | model.layers.0.attn          | MultiHeadAttention        | 361 K 
9  | model.layers.0.attn.q_linear | Linear                    | 90 K  
10 | model.layers.0.attn.v_linear | Linear 

1

In [14]:
trainer.test()

--------------------------------------------------------------------------------
TEST RESULTS
{'avg_test_loss': tensor(1.4762, device='cuda:0'),
 'avg_val_loss': tensor(1.4762, device='cuda:0'),
 'test_acc': tensor(0.9541, device='cuda:0'),
 'test_loss': tensor(1.4762, device='cuda:0')}
--------------------------------------------------------------------------------

